# **Проект**

В данном проекте я использовал [датасет с платформы Kaggle](https://www.kaggle.com/zynicide/wine-reviews). В нём
для каждого wine хранится его описание, цена, место происхождения, сорт и другие параметры

winemag-data_first150k.csv &mdash; Датасет

description.pdf &mdash; описание проекта

Расшифровка столбцов в winemag-data_first150k.csv для справки:

- country &mdash; страна производства

- description &mdash; описание продукта (wine)

- price &mdash; цена

- province &mdash; место производства

- region &mdash; регион производства

- variety &mdash; сорт


In [ ]:
import pandas as pd
import numpy as np
from joblib import dump, load
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/winemag-data_first150k.csv")
df

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude
...,...,...,...,...,...,...,...,...,...,...,...
150925,150925,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio
150926,150926,France,"Offers an intriguing nose with ginger, lime an...",Cuvée Prestige,91,27.0,Champagne,Champagne,NaN,Champagne Blend,H.Germain
150927,150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Terredora
150928,150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset


In [ ]:
df.isnull().sum()

Unnamed: 0         0
country            5
description        0
designation    45735
points             0
price          13695
province           5
region_1       25060
region_2       89977
variety            0
winery             0
dtype: int64

In [ ]:
df.drop(df[df["price"].isnull()].index,inplace=True)
df.drop(df[df["region_1"].isnull()].index,inplace=True)

In [ ]:
df.isnull().sum()
len(df)

114393

In [ ]:
from collections import Counter

df.drop(["Unnamed: 0","country","designation","province", "region_2","winery"],axis=1,inplace=True)
def compute(tmp):
    my_list = []
    for tmp2 in tmp:
        for s in tmp2.split():
            my_list.append(s)
    my_map = Counter(my_list)
    my_array = [element[0] for element in my_map.most_common(1500)]
    description = []
    for tmp2 in tmp:
        description_of_element = np.zeros(len(my_array))
        new_set = set(tmp2.split())
        for index, s in enumerate(my_array):
            if s in new_set:
                description_of_element[index] = 1
        description.append(description_of_element)
    return description

df["description"] = compute(df["description"])
df["region_1"] = compute(df["region_1"])
df["variety"] = compute(df["variety"])
df

,description,points,price,region_1,variety
0,"[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...",96,235.0,"[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...",96,110.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...",96,90.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3,"[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...",96,65.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,"[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",95,66.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...
150925,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",91,20.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
150926,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...",91,27.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
150927,"[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",91,20.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
150928,"[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",90,52.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
X_data = []
max_price = df["price"].max()
for index, tmp in df.iterrows():
  X_data.append(np.concatenate((tmp["description"], tmp["region_1"], tmp["variety"], [tmp["price"] / max_price])))
X_data = np.array(X_data)
Y_data = []
max_ans = df["points"].max()
for index, tmp in df.iterrows():
  Y_data.append(tmp["points"] / max_ans)
Y_data = np.array(Y_data)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X_data,Y_data,test_size=0.3)

In [ ]:
X_data = None
Y_data = None
df = None
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [ ]:
reg = LinearRegression()
reg.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
test_data_cnt = len(y_test)
pred = reg.predict(x_test)
pred = np.clip(pred, 0, 1)
answer = y_test

pred = pred * max_ans
answer = answer * max_ans

diff = np.sum(np.abs(pred - answer))

print(diff / test_data_cnt)

1.4007282154948952


In [ ]:
dstree = DecisionTreeRegressor()
dstree.fit(x_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
pred = dstree.predict(x_test)
pred = np.clip(pred, 0, 1)
answer = y_test

pred = pred * max_ans
answer = answer * max_ans

diff = np.sum(np.abs(pred - answer))

print(diff / test_data_cnt)

1.1946208986537694
